# RG-C Check of Target Polarization
This is a quick check of the target polarization data that is computed with Nathan and Sergey's code in the DAQ. Not all the computed variables are stored in the MAYA database. What seems to be available is:

	• B_DAQ:HEL:60m:0:asy, asycorr    - Trigger bit 0 asymmetry, corrected for beam asym
	• B_DAQ:HEL:60m:29:asy, asycorr  - Trigger bit 29 asym, corrected for beam asym
	• B_DAQ:HEL:60m:0:easy, easycorr
	• B_DAQ:HEL:60m:29:easy, easycorr
	• B_DAQ:HEL:60m:fcup:asy             -- FCup beam asymeetry
	• B_DAQ:HEL:60m:29:pol                  -- Polarization calculated by calibrating "29"
	• B_DAQ:HEL:30m:0:asy, asycorr
	• B_DAQ:HEL:30m:29:asy, asycorr
	• B_DAQ:HEL:30m:0:easy, easycorr
	• B_DAQ:HEL:30m:29:easy, easycorr
	• B_DAQ:HEL:30m:fcup:asy             -- FCup beam asymeetry
	• B_DAQ:HEL:30m:29:pol                  -- Target NMR Polarization
	• B_DAQ:HEL:10m:0:asy
	• B_DAQ:HEL:10m:29:asy
	• B_DAQ:HEL:10m:fcup:asy
	• B_DAQ:HEL:2m:0:asy
	• B_DAQ:HEL:2m:29:pol
	• B_DAQ:HEL:fcup:asy

The difficulty for extracting the average polarization over the duration of a run is that the 60m and 30m data samples, which are rolling averages over 60m or 30m respectively,
do not allow to closely match the start and end time of the run. The 10m and 2m averages do not have the beam charge asymmetry corrected values saved, though in theory this can be reconstructed from the "fcup:asy" channel.

Furthermore, the average of these samples is not what we actually want to compare with, but without the actual counts stored we cannot reconstruct the actual asymmetry we want. That information *is* containted in the data,
so can be obtained from a data analysis. In other words:
$$ A = \frac{N^+ - N^-}{N^+ - N^-} = \frac{N_1^+ - N_1^- + N_2^+ - N_2^- \ldots N_n^+ - N_n^-}{N_1^+ - N_1^- + N_2^+ - N_2^- \ldots N_n^+ - N_n^-} \ne
\frac{1}{n}\left(\frac{N_1^+ - N_1^-}{N_1^+ - N_1^-} + \frac{N_2^+ - N_2^-}{N_2^+ - N_2^-} \ldots \frac{N_n^+ - N_n^-}{N_n^+ - N_n^-}\right)$$

This is then further complicated in that the asymmetry is recorded every minute integrating over the last 60m (or 30m, 10m), so there will be double counting if you simply average the data.

## Step 1: Make a plot of the different channels.

In [1]:
import numpy as np

import RunData.MyaData as Mya
from RGC2022 import *
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default="plotly_white"
pio.renderers.default = 'jupyterlab'  # Alternate: 'browser' opens new browser window with plots.
from datetime import datetime
data = RunData(cache_file="RGC_2022_targ_pol.sqlite3", i_am_at_jlab=False)

RGC setup.
Creating the run data cache.


### Setup a time span for plotting

In [2]:
start = datetime(2022, 10, 10, 12, 0)
end = datetime(2022, 10, 20, 5, 0) # datetime.now() # datetime(2022, 10, 16, 12, 0)
setup_rundata_structures(data, (start, end))

Fetching the data from 2022-10-10 12:00:00 to 2022-10-20 05:00:00


### Grab the data from MYA for the specified time span

In [3]:
trigger_asym = data.Mya.get("B_DAQ:HEL:60m:29:asycorr", start, end)
trigger_asym_err = data.Mya.get("B_DAQ:HEL:60m:29:easycorr", start, end)
trigger_pol = data.Mya.get("B_DAQ:HEL:60m:29:pol", start, end)
fcup = data.Mya.get("B_DAQ:HEL:60m:fcup:asy", start, end)
fcup_error = data.Mya.get("B_DAQ:HEL:60m:fcup:easy", start, end)
trigger_asym30 = data.Mya.get("B_DAQ:HEL:30m:29:asycorr", start, end)
trigger_asym30_err = data.Mya.get("B_DAQ:HEL:30m:29:easycorr", start, end)
trigger_pol30 = data.Mya.get("B_DAQ:HEL:30m:29:pol", start, end)
fcup30 = data.Mya.get("B_DAQ:HEL:60m:fcup:asy", start, end)
fcup30_error = data.Mya.get("B_DAQ:HEL:60m:fcup:easy", start, end)

In [4]:
# Compute the error bands
x = trigger_asym["time"]
y = trigger_asym["value"]
xp = trigger_pol["time"]
yp = trigger_pol["value"]
x30 = trigger_asym30["time"]
y30 = trigger_asym30["value"]
xp30 = trigger_pol30["time"]
yp30 = trigger_pol30["value"]
# Get the errors at the correct time values by interpolating
y_err = np.interp(trigger_asym["time"], trigger_asym_err["time"], trigger_asym_err["value"])
y30_err = np.interp(trigger_asym30["time"], trigger_asym30_err["time"], trigger_asym30_err["value"])
#
# Get the "Harut scaling values" - the values used to convert raw asym to target pol.
yp_interp = np.interp(trigger_asym["time"], trigger_pol["time"], trigger_pol["value"])  # Interpolate
yp_ratio  = yp_interp/trigger_asym["value"]
yp_interp30 = np.interp(trigger_asym30["time"], trigger_pol30["time"], trigger_pol30["value"])  # Interpolate
yp_ratio30  = yp_interp30/trigger_asym30["value"]
trial_values = (42, 34)
correction_factors = np.zeros_like(yp_ratio)
correction_factors30 = np.zeros_like(yp_ratio30)
for tr in trial_values:
    correction_factors += np.average(yp_ratio[np.abs(yp_ratio - tr) < 1.])*(np.abs(yp_ratio - tr) < 1.)
    correction_factors30 += np.average(yp_ratio30[np.abs(yp_ratio30 - tr) < 1.])*(np.abs(yp_ratio30 - tr) < 1.)
#
# "Fold" the data
#
x = np.append(x, x[::-1])   # X and then X backward
y_err = np.append(y + y_err, y[::-1] - y_err[::-1])* np.append(correction_factors, correction_factors[::-1])  # Upper and then lower limit.
x30 = np.append(x30, x30[::-1])   # X and then X backward
y30_err = np.append(y30 + y30_err, y30[::-1] - y30_err[::-1])*np.append(correction_factors30, correction_factors30[::-1])  # Upper and then lower limit.
#

In [5]:
# For displaying the runs, to make it clear when they are.
runs = data.All_Runs.copy()
runs["center"] = runs.start_time + (runs.end_time - runs.start_time)/2
runs["dt"] = (runs.end_time - runs.start_time).map(lambda x: x.total_seconds()*1000)
runs.drop(runs.loc[ (runs.dt <= 0) ].index, inplace=True)
runs["event_rate"] = runs.event_count/runs.dt
runs["hover"] = [f"Run: {r}<br />"
                     f"Target:{runs.loc[r, 'target']}<br />"
                     f"Start: {runs.loc[r, 'start_time']}<br />"
                     f"End: {runs.loc[r, 'end_time']}<br />"
                     f"DT:   {runs.loc[r, 'dt'] / 1000.:5.1f} s<br />"
                     f"NEvt: {runs.loc[r, 'event_count']:10,d}<br />"
                     f"Target Pol: {runs.loc[r, 'target_polarization']}<br />"
                     f"Half W Plate: {runs.loc[r, 'half_wave_plate']} <br />"
                     f"Charge: {runs.loc[r, 'charge']:6.2f} mC <br />"
                     f"Lumi: {runs.loc[r, 'luminosity']:6.2f} 1/fb<br />"
                     f"<Rate>:{runs.loc[r, 'event_rate']:6.2f}kHz<br />"
                     for r in runs.index]


In [6]:
fig = make_subplots(rows=1, specs=[[{"secondary_y": True}]])
fig.update_layout(width=1800,  height=900)
fig.add_trace( go.Scatter(x=trigger_pol["time"], y=trigger_pol["value"], name="Pol 60m", line=dict(color="blue")))
fig.add_trace( go.Scatter(x=trigger_pol30["time"], y=trigger_pol30["value"], name="Pol, 30m", line=dict(color='rgba(0,150,255,1.)')))
fig.add_trace( go.Scatter(x=trigger_asym["time"], y=trigger_asym["value"]*correction_factors, name="Bit 29 asym 60m * cor_factor", line=dict(color="red")))
fig.add_trace( go.Scatter(x=x, y= y_err, name="Bit 29 error band 60m",
                          fill='toself', fillcolor='rgba(200,100,100,0.2)', line=dict(color='rgba(255,255,255,0)'), hoverinfo="skip", showlegend=False))
fig.add_trace( go.Scatter(x=trigger_asym30["time"], y=trigger_asym30["value"]*correction_factors30, name="Bit 29 asym 30m * cor_factor", line=dict(color="green")) )
fig.add_trace( go.Scatter(x=x30, y= y30_err, name="Bit 29 error band 30m",
                          fill='toself', fillcolor='rgba(100,200,100,0.2)', line=dict(color='rgba(255,255,255,0)'), hoverinfo="skip", showlegend=False))
fig.add_trace(go.Bar(x=runs.center, y=runs.event_rate, width=runs.dt, hovertext=runs.hover, name="Event rate", marker=dict(color='rgba(200,200,250,0.2)') ), row=1, col=1, secondary_y=True)
fig.update_yaxes(title_text="<b>Target Pol</b>",
                 titlefont=dict(size=22),
                 range=[-1.2, 1.2],
                 secondary_y=False,
                 tickfont=dict(size=18)
                 )

fig.update_yaxes(title_text="<b>ave. current.</b>",
                 titlefont=dict(size=14),
                 secondary_y=True,
                 tickfont=dict(size=12)
                 )


fig.update_xaxes(
            title_text="Date",
            titlefont=dict(size=22),
            tickfont=dict(size=18),
        )

fig.show()

### Compute Averages
Compute averages in different ways and compare the resuling values.

1. Take the first sample 60m (30m) after the start of the run, add additional samples every 60m (30m) until the sample is after the end of the run. (too much data at the end).
   * The last sample needs to be checked. If it is too far from the previous sample, there may be a half-wave plate change, or target change, or target pol change, in which case we discard that sample.
2. Take the first sample 60m (30m) after the start of the run, add additional samples at +60m (30m) only if these samples fit before the end of run. (not enough data at the end).
3. Take all the samples between start and end of run. (double counting, more data at start and end)

The construct `trigger_asym['time'].sub(time_to_get).abs().argmin()` is a quick way to get the index in the `trigger_asym['time']` closest to the time `time_to_get`.

In [7]:
def method1_indexes(run_number, asym, dt):
    indx_out = []
    check_time = runs.loc[run_number,'start_time']
    i=1
#    print("Method 1")
    while True:
        find_time = runs.loc[run_number,'start_time']+ timedelta(minutes=dt*i)
        if find_time > asym.iloc[-1].time:  # Past the end of available data.
            break
        idx = asym['time'].sub(find_time).abs().argmin()
        check_time = asym.iloc[idx].time
        indx_out.append(idx)
#        print(i, idx, check_time)
        if check_time > runs.loc[run_number,'end_time']:
            break
        i += 1
    return indx_out

def method1_unmod(run_number, asym, dt):
    indexes = method1_indexes(run_number, asym, dt)
    if len(indexes) > 0:
        return np.average(asym.iloc[indexes].value)
    else:
        return 0

def method1(run_number, asym, dt):
    indexes = method1_indexes(run_number, asym, dt)
    if len(indexes)>1 and np.abs(asym.iloc[indexes[-1]].value - asym.iloc[indexes[-2]].value) > 0.01:
        indexes.pop()
    if len(indexes) > 0:
        return np.average(asym.iloc[indexes].value)
    else:
        return 0

def method2_indexes(run_number, asym, dt):
    indx_out = []
    i=1
    check_time = runs.loc[run_number,'start_time']
#    print("Method 2")
    while True:
        find_time = runs.loc[run_number,'start_time']+ timedelta(minutes=dt*i)
        if find_time > asym.iloc[-1].time:  # Past the end of available data.
            break
        idx = asym['time'].sub(find_time).abs().argmin()
        check_time = asym.iloc[idx].time
        if check_time > runs.loc[run_number,'end_time']:
            break
        indx_out.append(idx)
#        print(i, idx, check_time)
        i += 1
    return indx_out

def method2(run_number, asym, dt):
    indexes = method2_indexes(run_number, asym, dt)
    if len(indexes) > 0:
        return np.average(asym.iloc[indexes].value)
    else:
        return 0

def method3_indexes(run_number, asym, dt=0):
     return range(asym['time'].sub(runs.loc[run_number,'start_time']).abs().argmin(), asym['time'].sub(runs.loc[run_number,'end_time']).abs().argmin())


def method3(run_number, asym, dt=0):
    indexes = method3_indexes(run_number, asym)
    if len(indexes) > 0:
        return np.average(asym.iloc[indexes].value)
    else:
        return 0

def target_pol_correction_factors(run_number):
    idx_start = trigger_asym30['time'].sub(runs.loc[run_number,'start_time']).abs().argmin()
    idx_end = trigger_asym30['time'].sub(runs.loc[run_number,'end_time']).abs().argmin()
    tra30 = trigger_asym30.iloc[idx_start:idx_end]
    tra30_clean = tra30.drop(tra30.loc[tra30.value.abs() <  0.0001].index)
    if len(tra30_clean) < 1:
        return 0
    yp_p30 = np.interp(tra30_clean["time"], trigger_pol30["time"], trigger_pol30["value"])  # Interpolate
    yp_r30  = yp_p30/tra30_clean["value"].to_numpy()
    return np.average(yp_r30)



indexes_method1 = method1_indexes(17191, trigger_asym, 60)
indexes_method2 = method2_indexes(17191, trigger_asym, 60)
indexes_method3 = method3_indexes(17191, trigger_asym, 60)

In [8]:
print(f"Method1 Average: {method1(17191, trigger_asym, 60)}")
print(f"Method2 Average: {method2(17191, trigger_asym, 60)}")
print(f"Method3 Average: {method2(17191, trigger_asym, 60)}")
ave1 = np.average(trigger_asym.iloc[indexes_method1].value, weights=np.interp(trigger_asym.iloc[indexes_method1]['time'], trigger_asym_err['time'], trigger_asym_err['value']))
std1 = np.average( (trigger_asym.iloc[indexes_method1].value -  ave1)**2, weights=np.interp(trigger_asym.iloc[indexes_method1]['time'], trigger_asym_err['time'], trigger_asym_err['value']))
ave2 = np.average(trigger_asym.iloc[indexes_method2].value, weights=np.interp(trigger_asym.iloc[indexes_method2]['time'], trigger_asym_err['time'], trigger_asym_err['value']))
std2 = np.average( (trigger_asym.iloc[indexes_method2].value -  ave2)**2, weights=np.interp(trigger_asym.iloc[indexes_method2]['time'], trigger_asym_err['time'], trigger_asym_err['value']))
ave3 = np.average(trigger_asym.iloc[indexes_method3].value, weights=np.interp(trigger_asym.iloc[indexes_method3]['time'], trigger_asym_err['time'], trigger_asym_err['value']))
std3 = np.average( (trigger_asym.iloc[indexes_method3].value -  ave3)**2, weights=np.interp(trigger_asym.iloc[indexes_method3]['time'], trigger_asym_err['time'], trigger_asym_err['value']))
print(f"Method1 Average: {ave1} +/- {std1}")
print(f"Method1 Average: {ave2} +/- {std2}")
print(f"Method1 Average: {ave3} +/- {std3}")


Method1 Average: 0.0188392
Method2 Average: 0.01884025
Method3 Average: 0.01884025
Method1 Average: 0.018834637640889905 +/- 2.8041329874036414e-07
Method1 Average: 0.018834547321577782 +/- 3.5030728161907616e-07
Method1 Average: 0.019125701436791917 +/- 8.782714696279642e-07


It seems that the weighting and computing the standard deviation are not particularly useful for our situation. So I will continue with the simpler method.

In [9]:
for rn in runs.index:
    print(f"Processing run {rn}")
    runs.loc[rn,"targ_pol_corr_factor"] = target_pol_correction_factors(rn)
    runs.loc[rn,"targ_pol60_method1"] = method1(rn, trigger_asym, 60)
    runs.loc[rn,"targ_pol60_method2"] = method2(rn, trigger_asym, 60)
    runs.loc[rn,"targ_pol60_method3"] = method3(rn, trigger_asym, 60)
    runs.loc[rn,"targ_pol30_method1"] = method1(rn, trigger_asym30, 30)
    runs.loc[rn,"targ_pol30_method2"] = method2(rn, trigger_asym30, 30)
    runs.loc[rn,"targ_pol30_method3"] = method3(rn, trigger_asym30, 30)


Processing run 17153
Processing run 17155
Processing run 17156
Processing run 17157
Processing run 17158
Processing run 17159
Processing run 17160
Processing run 17161
Processing run 17162
Processing run 17163
Processing run 17164
Processing run 17165
Processing run 17166
Processing run 17167
Processing run 17168
Processing run 17169
Processing run 17170
Processing run 17171
Processing run 17172
Processing run 17173
Processing run 17177
Processing run 17178
Processing run 17179
Processing run 17180
Processing run 17181
Processing run 17182
Processing run 17183
Processing run 17185
Processing run 17187
Processing run 17188
Processing run 17189
Processing run 17190
Processing run 17191
Processing run 17192
Processing run 17194
Processing run 17195
Processing run 17197
Processing run 17199
Processing run 17201
Processing run 17202
Processing run 17203
Processing run 17204
Processing run 17206
Processing run 17207
Processing run 17211
Processing run 17212
Processing run 17213
Processing ru

In [10]:
runs.loc[:,["start_time","target","targ_pol_corr_factor", 'targ_pol60_method1', 'targ_pol60_method2', 'targ_pol60_method3', 'targ_pol30_method1', 'targ_pol30_method2', 'targ_pol30_method3']]

,start_time,target,targ_pol_corr_factor,targ_pol60_method1,targ_pol60_method2,targ_pol60_method3,targ_pol30_method1,targ_pol30_method2,targ_pol30_method3
number,,,,,,,,,
17153,2022-10-10 12:11:00,NH3,42.296589,0.020268,0.020010,0.013095,0.021308,0.020708,0.018027
17155,2022-10-10 15:39:32,NH3,42.285707,0.019230,0.019415,0.020043,0.019366,0.019456,0.019855
17156,2022-10-10 19:42:34,NH3,42.277819,0.019642,0.019737,0.019446,0.019741,0.019798,0.019481
17157,2022-10-10 23:49:10,NH3,42.282118,0.019340,0.019244,0.019560,0.019275,0.019322,0.019564
17158,2022-10-11 02:48:21,NH3,42.279631,0.019284,0.019249,0.019222,0.019376,0.019256,0.019133
17159,2022-10-11 05:46:39,NH3,42.304728,0.019157,0.000000,0.019258,0.019252,0.000000,0.019731
17160,2022-10-11 05:53:21,NH3,42.290898,0.019233,0.000000,0.019304,0.019030,0.000000,0.019225
17161,2022-10-11 06:22:36,NH3,42.279820,0.018719,0.018719,0.018942,0.019358,0.019358,0.018884
17162,2022-10-11 11:50:11,NH3,34.727282,-0.022291,-0.022214,-0.021655,-0.022487,-0.022302,-0.021910


In [12]:
fig = make_subplots(rows=1, specs=[[{"secondary_y": True}]])
fig.update_layout(width=1800,  height=900)
fig.add_trace( go.Scatter(x=trigger_pol["time"], y=trigger_pol["value"], name="Pol 60m", line=dict(color="blue")))
fig.add_trace( go.Scatter(x=trigger_pol30["time"], y=trigger_pol30["value"], name="Pol, 30m", line=dict(color='rgba(0,150,255,1.)')))
fig.add_trace( go.Scatter(x=runs.center, y= runs.targ_pol60_method1*runs.targ_pol_corr_factor, name="Method1 60m",mode='markers', marker=dict(color="red", size=10)))
fig.add_trace( go.Scatter(x=runs.center, y= runs.targ_pol30_method1*runs.targ_pol_corr_factor, name="Method1 30m",mode='markers', marker=dict(color="orange", size=10)))
fig.add_trace(go.Bar(x=runs.center, y=runs.event_rate, width=runs.dt, hovertext=runs.hover, name="Event rate", marker=dict(color='rgba(200,200,250,0.2)') ), secondary_y=True)
fig.update_yaxes(title_text="<b>Target Pol</b>",
                 titlefont=dict(size=22),
                 range=[-1.2, 1.2],
                 secondary_y=False,
                 tickfont=dict(size=18)
                 )

fig.update_yaxes(title_text="<b>ave. current.</b>",
                 titlefont=dict(size=14),
                 secondary_y=True,
                 tickfont=dict(size=12)
                 )


fig.update_xaxes(
            title_text="Date",
            titlefont=dict(size=22),
            tickfont=dict(size=18),
        )

fig.show()